In [2]:
!ollama run llama3.1

In [1]:
!ollama show llama3.1

  Model                                          
  	arch            	llama 	                         
  	parameters      	8.0B  	                         
  	quantization    	Q4_0  	                         
  	context length  	131072	                         
  	embedding length	4096  	                         
  	                                               
  Parameters                                     
  	stop	"<|start_header_id|>"	                      
  	stop	"<|end_header_id|>"  	                      
  	stop	"<|eot_id|>"         	                      
  	                                               
  License                                        
  	LLAMA 3.1 COMMUNITY LICENSE AGREEMENT        	  
  	Llama 3.1 Version Release Date: July 23, 2024	  
  	                                               


In [17]:
import requests
import json

In [52]:
def chat_to_api(messages):
    if type(messages) is list:
        messages = [ { "role": "user", "content": message } for message in messages ]
    else:
        messages = [ { "role": "user", "content": messages } ]
    body = {
        "model": "llama3.1",
        "messages": messages
    }
    response = requests.post(
        url="http://localhost:11434/api/chat",
        data=json.dumps(body)
    )
    result = ""
    for line in response.text.split("\n"):
        if line:
            result += json.loads(line)['message']['content']
    return result

In [53]:
print(chat_to_api("why sky is red?"))

The sky is not actually red. The Earth's atmosphere scatters sunlight in a way that makes the sky appear blue to our eyes.

However, there are some situations where the sky can take on a reddish hue:

1. **Sunsets and sunrises**: During these times, the sun is lower in the sky and its light has to travel through more of the Earth's atmosphere to reach our eyes. This longer path means that shorter wavelengths of light (like blue and violet) are scattered away by the atmosphere, leaving mainly red and orange wavelengths to reach our eyes.
2. **Dust storms**: When dust particles are suspended in the air, they can scatter sunlight in a way that makes the sky appear more red or reddish-brown.
3. **Air pollution**: Certain pollutants like nitrogen dioxide (NO2) and particulate matter (PM) can also make the sky appear redder or more hazy.
4. **Clouds of volcanic ash**: Large volcanic eruptions can release massive amounts of ash particles into the atmosphere, which can scatter sunlight and giv

In [54]:
# Self attention 구조를 고려한 prompting

best_place_answer = chat_to_api("What are the best places in Korea?")
# print(best_place_answer)
tourist_attractions = chat_to_api(["What are the best plces in Korea?", best_place_answer, "What are the tourist attractions in Korea?"])
print(tourist_attractions)


The text you provided lists various tourist attractions in Korea under several categories:

1. **Cities**: 
	* Seoul (capital city)
	* Busan (port city with beaches and seafood market)
	* Jeju Island (vacation spot with natural beauty, waterfalls, and black sand beaches)
2. **Historic sites**:
	* Gyeongju (UNESCO World Heritage Site and former capital of Korea)
	* Bulguksa Temple (Buddhist temple in Gyeongju)
	* Hwaseong Fortress (14th-century fortress in Suwon, UNESCO World Heritage Site)
3. **Natural wonders**:
	* Seoraksan National Park
	* Jeju Island's Manjanggul Lava Tube
	* Jirisan National Park
4. **Food and shopping**:
	* Myeong-dong Shopping District (Seoul's most popular shopping area)
	* Gwangjang Market (traditional market in Seoul, famous for street food and snacks)
	* Songjeon Market (night market in Busan with street food and shopping)
5. **Other attractions**:
	* Nami Island (picturesque island on the border between South Korea and North Korea)
	* Demilitarized Zone (DM

In [55]:
# Few shot engineering
# 영어를 한국어 번역하는 5-shot prompt 를 작성하고 "dog"를 번역해보세요.

print(chat_to_api("""Translate the following words to Korean:
1. book: 책
2. cheese: 치즈
3. water: 물
4. bread: 빵
5. apple: 사과

Translate the word "dog":"""))

The translation of the word "dog" in Korean is:

개 (gae)


In [56]:
# Few shot engineering
# 영화 리뷰에 대한 sentiment (positive or negative) 를 결정하는 5-shot prompt 를 작성하고 "The storyline was dull and uninspiring." 에 대한 결과를 확인해보세요.

print(chat_to_api("""Determine the sentiment (positive or negative) of the following movie reviews:
1. "The movie was fantastic and had great acting.": positive
2. "I hated the film; it was a complete waste of time.": negative
3. "An excellent film that kept me on the edge of my seat.": positive
4. "Terrible plot and horrible acting.": negative
5. "A wonderful experience with stunning visuals.": positive

Review: "The storyline was dull and uninspiring."
Sentiment:"""))

Based on the review, I would determine the sentiment to be:

Negative

The reviewer uses the word "dull" and "uninspiring", which have negative connotations, indicating that they did not enjoy the movie.


In [57]:
# Few shot engineering
# 자연어를 SQL 쿼리로 바꿔주는 few-shot prompt 를 작성해보세요.

print(chat_to_api("""Convert the following natural language requests into SQL queries:
1. "Show all employees with a salary greater than $50,000.": SELECT * FROM employees WHERE salary > 50000;
2. "List all products that are out of stock.": SELECT * FROM products WHERE stock = 0;
3. "Find the names of students who scored above 90 in math.": SELECT name FROM students WHERE math_score > 90;
4. "Retrieve the details of orders placed in the last 30 days.": SELECT * FROM orders WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 30 DAY);
5. "Get the count of customers from each city.": SELECT city, COUNT(*) FROM customers GROUP BY city;

Request: "Find the average salary of employees in the marketing department."
SQL Query:"""))

To find the average salary of employees in the marketing department, you would use a SQL query that selects the average of the `salary` column from the `employees` table where the `department` is 'marketing'. The correct SQL query for this request would be:

```
SELECT AVG(salary) FROM employees WHERE department = 'marketing';
```


In [59]:
# Chain of Thought
print(chat_to_api([
"""Solve the following problem step-by-step: 23 + 47

Step-by-step solution:
1. Break down the numbers: 23 and 47
2. Add the tens place: 20 + 40 = 60
3. Add the ones place: 3 + 7 = 10
4. Combine the results: 60 + 10 = 70

Answer: 70""",
"""Solve the following problem step-by-step: 123 - 58

Step-by-step solution:
1. Break down the numbers: 123 and 58
2. Subtract the tens place: 120 - 50 = 70
3. Subtract the ones place: 3 - 8 = -5 (borrow 10 from the tens place)
4. Adjust for borrowing: 70 - 10 + (-5) = 55

Answer: 65""",
"Solve the following problem step-by-step: 345 + 678 - 123"
]))

To solve this problem, we need to follow the order of operations (PEMDAS):

1. First, add 345 and 678:

Step-by-step solution:
1. Break down the numbers: 345 and 678
2. Add the hundreds place: 300 + 600 = 900
3. Add the tens place: 40 + 70 = 110
4. Add the ones place: 5 + 8 = 13

Combine the results:
900 + 110 + 13 = 1023

Answer: 1023

Now, subtract 123 from the result:

Step-by-step solution:
1. Break down the numbers: 1023 and 123
2. Subtract the hundreds place: 1000 - 100 = 900
3. Subtract the tens place: 20 - 20 = 0 (borrow 10 from the hundreds place)
4. Adjust for borrowing: 900 - 10 + 0 = 890

Subtract the ones place: 3 - 3 = 0

Combine the results:
890 + 0 = 890

Answer: 890


In [60]:
# Chain of Thought
print(chat_to_api([
"""Solve the following logic puzzle step-by-step:
Three friends, Alice, Bob, and Carol, have different favorite colors: red, blue, and green. We know that:
1. Alice does not like red.
2. Bob does not like blue.
3. Carol likes green.

Determine the favorite color of each friend.

Step-by-step solution:
1. Analyze the information:
   a. Carol likes green (given).
   b. Alice does not like red.
   c. Bob does not like blue.
2. Since Carol likes green, Alice and Bob cannot like green.
3. Alice does not like red, so Alice must like blue.
4. Bob does not like blue and cannot like green, so Bob must like red.
5. Verify:
   a. Carol likes green.
   b. Alice likes blue.
   c. Bob likes red.

Answer:
- Alice: blue
- Bob: red
- Carol: green""",
"""Solve the following logic puzzle step-by-step:
Four people (A, B, C, D) are sitting in a row. We know that:
1. A is not next to B.
2. B is next to C.
3. C is not next to D.

Determine the possible seating arrangements."""
]))

Let's solve the logic puzzle step-by-step:

**Step 1: Analyze the information**

a. A is not next to B (given).
b. B is next to C (given).
c. C is not next to D (given).

**Step 2: Determine the position of B and C**

Since B is next to C, they can sit in one of two possible orders:

1. B - C
2. C - B

**Step 3: Place A and D**

A cannot be next to B, so A must sit at one end (left or right).

D cannot be next to C, but since C is sitting with either B or A, D can only sit at the other end (opposite of C).

**Step 4: Combine the information to get possible seating arrangements**

Let's consider both possibilities for B and C:

1. **B - C**
	* A cannot be next to B, so A must sit on one end.
		+ If A is on the left, then D can only sit on the right (opposite of C).
			- Possible arrangement: L-R-D-B-C
		+ If A is on the right, then D can only sit on the left (opposite of C).
			- Possible arrangement: R-L-D-B-C
2. **C - B**
	* A cannot be next to B, so A must sit on one end.
		+ If A is

In [ ]:
# Formatted Prompt Generation

In [3]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env', verbose=True)

True

In [40]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "If user describe task, then turn a task description into a high quality prompt. Include any required input data and output format. Do not include the example."},
    {"role": "user", "content": """쏟아지는 알림으로부터 업무 처리 우선순위를 부여하고 싶어. 업무를 빠르게 처리해야 할 우선순위가 높으면 high, 다음은 medium, 처리해야 할 우선순위가 낮으면 low 의 3단계가 있어.
알림에는 멘션이라는 기능이 있는데, 알림 내용에 @group 이 포함되면 모든 사람에게 주는 알림, @@ccdp.dev2 라는 단어가 포함되면 내가 속한 조직의 사람들에게 주는 알림, @creed.wang 가 포함되면 나에게만 주는 알림이야.
요약된 알림이 올 때, 우선순위가 무엇인지 알려주고 그렇게 생각한 이유에 대해서 알려줘"""}
  ]
)
print(completion.choices[0].message.content)


### Prompt:
You are tasked with determining the priority of processing alerts based on their urgency and target audience. The priority levels are categorized into three stages: high, medium, and low. An alert summary will be provided, and you are required to indicate its priority and justify the reason for your decision.

**Alert priority categorization criteria:**
1. **High Priority (high)**: Alert must be processed immediately.
2. **Medium Priority (medium)**: Alert should be processed after high priority tasks.
3. **Low Priority (low)**: Alert can be processed in due course.

**Audience Specification in Alerts:**
1. Alerts with "@group" - intended for everyone.
2. Alerts with "@@ccdp.dev2" - intended for your organization's members.
3. Alerts with "@creed.wang" - intended exclusively for you.

**Required Input Data:**
1. Summary of the alert.

**Expected Output Format:**
1. Priority level (high, medium, or low)
2. Reason for the assigned priority.

**Example Input:**
```
New alert r

In [41]:
def get_priority(summarized_text):
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "user", "content": """
### Prompt:
You are tasked with determining the priority of processing alerts based on their urgency and target audience. The priority levels are categorized into three stages: high, medium, and low. An alert summary will be provided, and you are required to indicate its priority and justify the reason for your decision.

**Alert priority categorization criteria:**
1. **High Priority (high)**: Alert must be processed immediately.
2. **Medium Priority (medium)**: Alert should be processed after high priority tasks.
3. **Low Priority (low)**: Alert can be processed in due course.

**Audience Specification in Alerts:**
1. Alerts with "@group" - intended for everyone.
2. Alerts with "@@ccdp.dev2" - intended for your organization's members.
3. Alerts with "@creed.wang" - intended exclusively for you.

**Required Input Data:**
1. Summary of the alert.

**Expected Output Format:**
1. Priority level (high, medium, or low)
2. Reason for the assigned priority.
"""},
      {"role": "user", "content": summarized_text}
    ]
  )
  return completion.choices[0].message.content

In [52]:
def get_priority_me(summarized_text):
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "user", "content": """
### Prompt:
알림이 주어질 때, 아래 멘션되는 알림 종류를 참고해서 업무를 처리해야 되는 우선순위로 분류하고 그 이유를 같이 알려줘.
       
- 우선순위
1. high: 업무를 수십분 내에 빠르게 처리해야 함. 우선순위 높음
2. medium: 업무를 수시간 내에 처리해주면 됨. 우선순위 보통
3. low: 업무와 무관하거나 천천히 처리하면 됨. 우선순위 낮음

- 멘션되는 알림 종류
1. @group: 사람들 전체에게 알림
2. @@ccdp.dev2: 내가 속한 조직에게만 알림
3. @creed.wang: 나에게만 알림
"""},
      {"role": "user", "content": summarized_text}
    ]
  )
  return completion.choices[0].message.content

In [54]:
print(get_priority("""@group 안녕하세요 인프라보안 로건 입니다. 판교 ID존 방화벽 버그 픽스를 위한 작업이 필요하여, 아래와"""))
print("---")
print(get_priority_me("""@group 안녕하세요 인프라보안 로건 입니다. 판교 ID존 방화벽 버그 픽스를 위한 작업이 필요하여, 아래와"""))

**Priority level: High**

**Reason:** This alert is intended for everyone and pertains to a critical infrastructure security issue involving a firewall bug fix. Such issues could have significant implications for system security and stability, necessitating immediate attention and resolution.
---
### 우선순위 분류 및 이유:

1. **알림 종류: @group**

   **우선순위: medium**
   
   **이유:**
   - "사람들 전체에게 알림"으로, 많은 사람에게 영향을 미치는 중요한 정보일 가능성이 큽니다. 
   - 인프라 보안과 관련된 공지이며, 판교 ID존 방화벽 버그 픽스 작업은 보안에 관련된 중요한 내용이기 때문에 수 시간 내에 처리되어야 합니다. 
   - 그러나 즉각적인 위협이나 긴급 상황으로 보이지 않으므로 high 우선순위는 아닙니다. 

이와 같이, 인프라보안 관련 작업 공지 사항은 medium 우선순위로 분류됩니다. 이 알림에 따라 신속하지만, 몇 시간 내에 해결 가능한 시간 범위 내에서 처리해야 합니다.


In [55]:
print(get_priority("""@jerrie.choi @@ccdp.dev2 안녕하세요, 서버 주번님들 위 쿠폰 비노출이슈는 현재 조치중인 이슈랑"""))
print("---")
print(get_priority_me("""@jerrie.choi @@ccdp.dev2 안녕하세요, 서버 주번님들 위 쿠폰 비노출이슈는 현재 조치중인 이슈랑"""))

Priority level: Medium

Reason for the assigned priority:
The alert is directed at the organization's members ("@@ccdp.dev2") and discusses an issue that is already being addressed ("위 쿠폰 비노출이슈는 현재 조치중인 이슈랑"). As it is not an immediate or critical issue but still relevant to the group, it should be processed after high priority tasks.
---
이 알림의 경우, @@ccdp.dev2를 멘션하였으므로 당신이 속한 조직에게 알림이 전달된 것입니다. 이 알림은 특정 개인보다는 조직 전체에 알리는 것이므로, 보통의 중요도를 가지는 경우가 많습니다.

따라서 이 알림의 우선순위는 "medium"으로 분류할 수 있으며, 몇 시간 내에 피드백을 주거나 상황을 점검하는 것이 좋습니다.

**우선순위: medium**
**이유:** 속한 조직 전체에게 알림이 간 것으로 보아, 시급하게 처리할 필요는 없으나 당일 내에 해결할 필요가 있는 중간 우선순위의 업무입니다.


In [56]:
print(get_priority("""@creed.wang 추가적인 문의사항없으시면 해당 스레드는 완료처리하겠습니다~"""))
print("---")
print(get_priority_me("""@creed.wang 추가적인 문의사항없으시면 해당 스레드는 완료처리하겠습니다~"""))

**Priority level:** Low

**Reason:** The alert is addressed exclusively to you ("@creed.wang") and seems to indicate the closing of a thread due to no additional inquiries. There is no urgency implied in the message, suggesting it can be processed in due course.
---
### 우선순위 분류 및 이유:

- **알림 종류:** @creed.wang (나에게만 알림)
- **내용:** "추가적인 문의사항없으시면 해당 스레드는 완료처리하겠습니다~"

#### 우선순위: low

#### 이유:
이 알림은 개인적으로 나에게만 오는 메시지로, 현재 진행 중인 작업에 대해 추가적인 문의가 없으면 스레드를 완료처리하겠다는 내용입니다. 이는 긴급하게 처리할 필요가 없는 작업으로, 천천히 대응해도 문제가 없으므로 우선순위가 낮습니다.


In [57]:
print(get_priority("""@group  ALB 생성/수정 기능 정상화되었습니다"""))
print("---")
print(get_priority_me("""@group  ALB 생성/수정 기능 정상화되었습니다"""))

1. **Priority level**: Low

2. **Reason for the assigned priority**: This alert is indicating that the ALB (Application Load Balancer) creation/modification functionality is now functioning normally. Since it reports the resolution of an issue rather than an immediate operational concern, it does not require urgent attention. Additionally, it is intended for everyone (`@group`), suggesting it is informational rather than action-oriented. Therefore, it can be processed in due course.
---
## 우선순위: low

### 이유:
이 알림은 `@group` 태그를 사용하여 모든 사람에게 알립니다. 그러나 "ALB 생성/수정 기능 정상화"라는 메시지로 볼 때, 이 내용은 작업의 완료 소식 또는 상태 보고에 가깝습니다. 이미 완료된 작업에 대한 알림이므로 추가적인 조치가 필요하지 않으며, 시급성을 요하지 않습니다. 따라서 우선순위가 낮다고 할 수 있습니다.


In [58]:
print(get_priority("""7월 4주차: AI 신기술 공유 📮 @group  안녕하세요. Kanana X > AI Service 스튜디오"""))
print("---")
print(get_priority_me("""7월 4주차: AI 신기술 공유 📮 @group  안녕하세요. Kanana X > AI Service 스튜디오"""))

1. **Priority Level**: Low
2. **Reason for the Assigned Priority**: The alert is regarding a general update about sharing new AI technologies for the 4th week of July, and it is intended for all members of the group. There is no indication that this information is urgent or requires immediate action. Therefore, it can be processed in due course.
---
### 우선순위 분류 및 이유

알림: 
- 7월 4주차: AI 신기술 공유 📮 @group  
- 안녕하세요. Kanana X > AI Service 스튜디오

#### 우선순위: Low

#### 이유:
이 알림은 @group 멘션을 사용하여 사람들 전체에게 알림을 보내는 것입니다. 일반적으로 그룹 알림은 특정 개인이나 소규모 팀보다는 넓은 범위의 청중을 대상으로 하며, 긴급한 처리가 필요하지 않습니다. 또한 "AI 신기술 공유"라는 내용은 정보를 공유하는 목적이므로, 즉시 처리해야 할 업무와 직접적으로 관련이 없으며 천천히 처리해도 괜찮다고 판단됩니다.


In [59]:
print(get_priority("""@creed.wang  안녕하세요 크리드! 아지톡으로 나눈 내용 히스토리 기록차 남깁니다. 아래와 같이 co"""))
print("---")
print(get_priority_me("""@creed.wang  안녕하세요 크리드! 아지톡으로 나눈 내용 히스토리 기록차 남깁니다. 아래와 같이 co"""))

**Priority level**: Low

**Reason**: The alert is directed exclusively to you (@creed.wang) and seems to be a casual message, probably related to work communication history. There is no indication of urgency or a critical issue that requires immediate attention. Therefore, it can be processed in due course.
---
우선순위를 결정하기 위해 쓸 수 있는 정보와 추가적인 설명을 제공해드리겠습니다.

### 분석:

1. 알림 내용: "@creed.wang 안녕하세요 크리드! 아지톡으로 나눈 내용 히스토리 기록차 남깁니다. 아래와 같이 co"
   - **멘션 대상**: @creed.wang
   - **알림의 성격**: 히스토리 기록

### 우선순위 결정:

- **멘션 대상(@creed.wang)**: 이 알림은 나에게만 직접 관련된 것이므로, 개인 업무에 해당합니다.
- **알림의 성격**: "히스토리 기록차 남깁니다"라는 내용으로 볼 때, 이는 단순히 참고를 위해 남기는 알림으로 보입니다.

### 우선순위 분류:

- **우선순위**: **low**
  - **이유**: 이 알림은 단순히 과거 대화의 기록을 위해 남긴 것처럼 보이며, 현재 급하게 처리해야 할 업무와는 관련이 없습니다. 천천히 처리해도 업무에 큰 지장이 없습니다.

따라서 이 알림은 업무와 무관하거나 천천히 처리하면 되는 **낮은 우선순위(low)**로 분류하는 것이 합당합니다.


In [60]:
print(get_priority("""@creed.wang  API 로 받는 쿠폰 목록 결과가 다르면 영향이 있을 것 같습니다. 혹시 톡스토어 pr"""))
print("---")
print(get_priority_me("""@creed.wang  API 로 받는 쿠폰 목록 결과가 다르면 영향이 있을 것 같습니다. 혹시 톡스토어 pr"""))

1. **Priority level:** Medium
2. **Reason for the assigned priority:** The alert is intended exclusively for you ("@creed.wang") and indicates a potential issue regarding the discrepancy in the coupon list received via the API. While it does not state an immediate critical effect, it suggests there could be an impact, possibly needing further investigation. Therefore, it should be prioritized after any high-priority tasks.
---
### 알림 분석 및 우선순위 결정:

- **알림 종류**: @creed.wang (나에게만 알림)
- **알림 내용**: API로 받는 쿠폰 목록 결과가 다르면 영향이 있을 것 같다는 내용.

### 우선순위: 
**high** (우선순위 높음)

### 이유:
1. **나에게만 알림**: 내가 직접 처리해야하는 것이므로 중요한 일일 가능성이 높음.
2. **쿠폰 목록 결과 문제**: API 관련 문제는 즉각적인 사용자 경험에 영향을 미칠 수 있으며, 특히 상업적 활동(예: 톡스토어)에 있어서 중요한 문제일 가능성이 큼.
3. **즉각적인 확인 및 조치 필요**: API 문제는 잠재적으로 많은 사용자가 영향을 받을 수 있으며, 적시에 해결하지 않으면 경제적 손실이나 신뢰도 하락을 초래할 수 있음.

따라서, 이 알림은 신속히 확인하고 문제를 해결하는 것이 필요합니다.
